In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split
import glob
import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean

from tqdm.notebook import tqdm
from pycaret.regression import *
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# 스마트팜 배출량에 대한 예측
- smart_farm_final.csv

# Pycaret을 이용한 튜닝 및 예측
 - 3 표준편차 기준 이상치 대체 반응변수
 - 왜도 3 이상 값들 로그변환 설명변수
 - VIF < 11 인 값들
 - 부스팅 / 배깅 / 블랜딩 3개 혼합모델

In [43]:
data = pd.read_csv("smart_farm_final.csv")
data.shape

(148168, 57)

In [44]:
data["환기온도(천창 제어온도)"].value_counts()

28    79025
30    41149
27    27994
Name: 환기온도(천창 제어온도), dtype: int64

In [45]:
data.iloc[:129448,:].tail(2)

,시간,년도,월,일,시,분,일사량(W/㎡),외기온(℃),외기습도(%),풍속(m/s),...,하루길이,시간범주,일몰전후,일출일몰,지면온도,오존등급,아황산가스등급,이산화질소등급,PM10등급,PM25등급
129446,2022-02-28 23:58:00,2022,2,28,23,58,0.0,4.1,52,0.4,...,1123,야간,일몰1시간반_후,일몰,0.7,좋음,좋음,좋음,좋음,보통
129447,2022-02-28 23:59:00,2022,2,28,23,59,0.0,4.1,52,0.4,...,1123,야간,일몰1시간반_후,일몰,0.7,좋음,좋음,좋음,좋음,보통


In [46]:
data.iloc[129448:,:]

,시간,년도,월,일,시,분,일사량(W/㎡),외기온(℃),외기습도(%),풍속(m/s),...,하루길이,시간범주,일몰전후,일출일몰,지면온도,오존등급,아황산가스등급,이산화질소등급,PM10등급,PM25등급
129448,2022-03-04 00:00:00,2022,3,4,0,0,0.0,2.0,76,0.0,...,1131,야간,일몰1시간반_후,일몰,-1.8,보통,좋음,좋음,보통,보통
129449,2022-03-04 00:01:00,2022,3,4,0,1,0.0,2.0,76,0.4,...,1131,야간,일몰1시간반_후,일몰,-1.8,보통,좋음,좋음,보통,보통
129450,2022-03-04 00:02:00,2022,3,4,0,2,0.0,1.9,76,0.0,...,1131,야간,일몰1시간반_후,일몰,-1.8,보통,좋음,좋음,보통,보통
129451,2022-03-04 00:03:00,2022,3,4,0,3,0.0,1.9,76,0.0,...,1131,야간,일몰1시간반_후,일몰,-1.8,보통,좋음,좋음,보통,보통
129452,2022-03-04 00:04:00,2022,3,4,0,4,0.0,1.9,76,0.0,...,1131,야간,일몰1시간반_후,일몰,-1.8,보통,좋음,좋음,보통,보통
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148163,2022-03-20 23:55:00,2022,3,20,23,55,0.0,6.2,61,0.4,...,1207,야간,일몰1시간반_후,일몰,4.6,보통,좋음,좋음,좋음,좋음
148164,2022-03-20 23:56:00,2022,3,20,23,56,0.0,6.2,61,0.0,...,1207,야간,일몰1시간반_후,일몰,4.6,보통,좋음,좋음,좋음,좋음
148165,2022-03-20 23:57:00,2022,3,20,23,57,0.0,6.1,61,0.4,...,1207,야간,일몰1시간반_후,일몰,4.6,보통,좋음,좋음,좋음,좋음
148166,2022-03-20 23:58:00,2022,3,20,23,58,0.0,6.1,61,0.4,...,1207,야간,일몰1시간반_후,일몰,4.6,보통,좋음,좋음,좋음,좋음


In [47]:
# 문자열 형식의 컬럼만 추출하기
object_lst = []

# 숫자 형식의 컬럼만 추출하기
num_lst = []

for col in data.columns:
    if data[col].dtype == "object":
        object_lst.append(col)
    
    else:
        num_lst.append(col)

#### 1. 숫자형 데이터 스케일링 진행
- 컬럼별로 스케일링 진행

In [48]:
data_scaled = data.copy()

# 날짜 관련 데이터는 스케일링 진행하지 않기 # 카테고리열 데이터
num_lst_nodate = [x for x in num_lst if x not in ["년도","월","일","시","분",
                                                  "배기팬","유동팬","팬코일 B동","환기온도(천창 제어온도)","열공급량(kWh)"]]

loglist=[]
# 왜도 3인 설명변수 log1p 변환
for val in num_lst_nodate:
    if abs(data_scaled[val].skew())>3:
        data_scaled[val] = np.log1p(data_scaled[val])
        loglist.append(val)
loglist


from sklearn.preprocessing import StandardScaler

for col in num_lst_nodate:
    std = StandardScaler()

    data_scaled[col] = std.fit_transform(data_scaled[col].values.reshape(-1,1))

"배기팬","유동팬","팬코일 B동","환기온도(천창 제어온도)" 카테고리열 데이터로 스케일링 필요없음

#### 1.1 데이터 처리하기
- 학습용 데이터 : ~2022.03
- 예측용 데이터 : 2022.03~

X 데이터 처리하기
- dummy 처리
- 왜도 처리

In [49]:
Y = data_scaled["열공급량(kWh)"]

X_dummy = pd.get_dummies(data_scaled.drop(["년월일","열공급량(kWh)"],axis=1)
                         ,columns=["년도"])

In [50]:
data_scaled_final = pd.concat([X_dummy, Y], axis = 1)

train_data = data_scaled_final.iloc[:129448,:]

test_data = data_scaled_final.iloc[129448:,:]

In [51]:
x_train = train_data.drop(["열공급량(kWh)"],axis=1)
y_train = train_data["열공급량(kWh)"]


x_test = test_data.drop(["열공급량(kWh)"],axis=1)
y_test = test_data["열공급량(kWh)"]

In [52]:
reg = setup(data=train_data,
                target='열공급량(kWh)',
                numeric_imputation = 'mean',
                normalize = False,
                normalize_method = 'zscore',
                categorical_features = ['차광스크린_광도조절',"환기온도(천창 제어온도)",
                                   '차광스크린_개방조절','차열스크린_온도조절','차열스크린_광도조절','차열스크린_개방조절',
                                   '시간범주','일몰전후','일출일몰','오존등급','아황산가스등급','이산화질소등급','PM10등급','PM25등급'],
                silent= True,
                fold_strategy = 'timeseries',
                fold = 3,
                session_id = 0,
                verbose=True)

,Description,Value
0,session_id,0
1,Target,열공급량(kWh)
2,Original Data,"(129448, 57)"
3,Missing Values,False
4,Numeric Features,36
5,Categorical Features,19
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(90613, 105)"


단일 모델 선택하기 위한 성능 비교

In [53]:
best_5_d = compare_models(sort = 'RMSLE', n_select = 5)
print(best_5_d)

[]
